# Data Owner (DO) - Bob's Notebook

This notebook demonstrates the **RemoteData architecture** from the data owner's perspective.

Bob owns sensitive patient data and wants to collaborate with Alice (a data scientist) while preserving privacy.

## Setup

In [1]:
# # Clean start
!cd shared && rm -rf alice public bob

In [2]:
import sys
sys.path.insert(0, "../python/src")

import beaver
import pandas as pd
import numpy as np
from beaver import Twin

In [3]:
# Connect as Bob (data owner)
bv = beaver.connect("shared", user="bob")

🔄 Auto-load replies enabled for bob (polling every 2.0s)


## 1. Create Real (Private) Data

Bob has sensitive patient data with real names, ages, and test results.

In [4]:
count = 1

In [5]:
counter = Twin(
    private=1,      # Real data (stays local)
    public=count,       # Mock data (shareable)
    owner="bob",
    name="count"
)

In [6]:
counter

⚠️  Twin: count (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    1    ← .value uses this
  🌍 Public    1    ✓
  Live: ⚫ Disabled
  IDs: twin=6bbb4062... private=c4e73fb5... public=7f9f9624...

In [7]:
counter.enable_live(interval=2.0)

🟢 Live sync enabled (read-only, every 2.0s)


In [8]:
counter

⚠️  Twin: count (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    1    ← .value uses this
  🌍 Public    1    ✓
  Live: 🟢 Enabled (read-only, 2.0s)
  Last sync: 2025-11-20T06:55:52.978774+00:00
  IDs: twin=6bbb4062... private=c4e73fb5... public=7f9f9624...

In [9]:
bv.remote_vars["counter"] = counter

🌍 Using PUBLIC data from Twin 'counter...'
📢 Published Twin 'counter' (public side available at: shared/public/bob/data/a11169e9560b4ccfa8f1f104741c55e0.beaver)


In [10]:
counter.public += 1

In [11]:
counter

⚠️  Twin: count (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    1    ← .value uses this
  🌍 Public    2    ✓
  Live: 🟢 Enabled (read-only, 2.0s)
  Last sync: 2025-11-20T06:55:52.978774+00:00
  IDs: twin=6bbb4062... private=c4e73fb5... public=7f9f9624...

In [12]:
for i in range(10):
    print(i)
    import time
    time.sleep(2)
    counter.public += 1

0
🌍 Using PUBLIC data from Twin 'counter...'
  📢 Re-published to public registry
1
🌍 Using PUBLIC data from Twin 'counter...'
  📢 Re-published to public registry
2
🌍 Using PUBLIC data from Twin 'counter...'
  📢 Re-published to public registry
3
🌍 Using PUBLIC data from Twin 'counter...'
  📢 Re-published to public registry
4
🌍 Using PUBLIC data from Twin 'counter...'
  📢 Re-published to public registry
5
🌍 Using PUBLIC data from Twin 'counter...'
  📢 Re-published to public registry
6
🌍 Using PUBLIC data from Twin 'counter...'
  📢 Re-published to public registry
7
🌍 Using PUBLIC data from Twin 'counter...'
  📢 Re-published to public registry
8
🌍 Using PUBLIC data from Twin 'counter...'
  📢 Re-published to public registry
9
🌍 Using PUBLIC data from Twin 'counter...'
  📢 Re-published to public registry


In [22]:
bv.workspace()

Name,Owner,Type,Status,Last Action
counter,bob,Twin[int],📤 Sent,3s ago


In [23]:
# Real patient data (SENSITIVE)
real_data = pd.DataFrame({
    'patient_id': ['P001', 'P002', 'P003', 'P004', 'P005', 'P006', 'P007', 'P008', 'P009', 'P010'],
    'name': ['Alice Johnson', 'Bob Smith', 'Carol White', 'David Brown', 'Eve Davis', 
             'Frank Miller', 'Grace Lee', 'Henry Wilson', 'Iris Taylor', 'Jack Anderson'],
    'age': [34, 45, 29, 52, 38, 41, 36, 48, 31, 55],
    'test_result': [7.2, 8.5, 6.1, 9.3, 7.8, 8.1, 6.9, 9.0, 7.5, 8.8],
    'diagnosis': ['positive', 'positive', 'negative', 'positive', 'positive', 
                  'positive', 'negative', 'positive', 'positive', 'positive']
})

print("🔒 PRIVATE DATA (Real Patient Records):")
real_data

🔒 PRIVATE DATA (Real Patient Records):


,patient_id,name,age,test_result,diagnosis
0,P001,Alice Johnson,34,7.2,positive
1,P002,Bob Smith,45,8.5,positive
2,P003,Carol White,29,6.1,negative
3,P004,David Brown,52,9.3,positive
4,P005,Eve Davis,38,7.8,positive
5,P006,Frank Miller,41,8.1,positive
6,P007,Grace Lee,36,6.9,negative
7,P008,Henry Wilson,48,9.0,positive
8,P009,Iris Taylor,31,7.5,positive
9,P010,Jack Anderson,55,8.8,positive


## 2. Create Mock (Public) Data

Bob creates anonymized mock data for Alice to develop her analysis on.

In [15]:
# Mock data - anonymized, smaller sample
mock_data = pd.DataFrame({
    'patient_id': ['M001', 'M002', 'M003', 'M004', 'M005'],
    'name': ['Patient A', 'Patient B', 'Patient C', 'Patient D', 'Patient E'],
    'age': [30, 40, 35, 50, 45],
    'test_result': [6.5, 8.0, 7.0, 9.0, 8.5],
    'diagnosis': ['negative', 'positive', 'positive', 'positive', 'positive']
})

print("🌍 PUBLIC DATA (Mock/Anonymized):")
mock_data

🌍 PUBLIC DATA (Mock/Anonymized):


,patient_id,name,age,test_result,diagnosis
0,M001,Patient A,30,6.5,negative
1,M002,Patient B,40,8.0,positive
2,M003,Patient C,35,7.0,positive
3,M004,Patient D,50,9.0,positive
4,M005,Patient E,45,8.5,positive


## 3. Create a Twin (Dual-Value Privacy Object)

A **Twin** holds both private (real) and public (mock) data.
- Bob sees and uses the **private** side locally
- When shared, only the **public** side is transmitted

In [16]:
# Create Twin with both sides
patient_data = Twin(
    private=real_data,      # Real data (stays local)
    public=mock_data,       # Mock data (shareable)
    owner="bob",
    name="patient_data"
)

patient_data

⚠️  Twin: patient_data (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private      patient_id           name  age  test_result diagnosis
0...    ← .value uses this
  🌍 Public      patient_id       name  age  test_result diagnosis
0    ...    ✓
  Live: ⚫ Disabled
  IDs: twin=1671f0e6... private=f0f04ee1... public=1701d52e...

In [17]:
bv.workspace()

Name,Owner,Type,Status,Last Action
counter,bob,Twin[int],📤 Sent,0s ago


## 4. Work with the Twin Locally

When Bob accesses `.value`, it uses the **private** side (real data).

In [18]:
# Bob works with real data
print(f"Total patients: {len(patient_data.value)}")
print(f"Average age: {patient_data.value['age'].mean():.1f}")
print(f"Positive rate: {(patient_data.value['diagnosis'] == 'positive').sum() / len(patient_data.value) * 100:.1f}%")

🔒 Using PRIVATE data from Twin 'patient_data...'
Total patients: 10
Average age: 40.9
Positive rate: 80.0%


In [19]:
# Notice: The "Using PRIVATE data" message only printed once!
# Multiple accesses don't spam the output
patient_data.value['test_result'].mean()

np.float64(7.92)

## 5. Share Twin with Data Scientist

When Bob sends the Twin to Alice, **only the public side is transmitted**.
The private data never leaves Bob's environment.

In [20]:
# Send to Alice - only public side is shared!
result = bv.send(patient_data, user="alice")
print(f"\n✉️  Sent to Alice: {result.path}")
print(f"📦 Envelope ID: {result.envelope.envelope_id[:12]}...")


✉️  Sent to Alice: shared/alice/a03df99a7904494bad20cddd0cff65da.beaver
📦 Envelope ID: a03df99a7904...


In [21]:
bv.workspace()

Name,Owner,Type,Status,Last Action
counter,bob,Twin[int],📤 Sent,0s ago


🌍 Using PUBLIC data from Twin 'counter...'
  📢 Re-published to public registry


In [24]:
bv.inbox()

name,id,sender,type,size_bytes,created_at,reply_to
real_result,45e80be31302...,alice,ComputationRequest,2173,2025-11-20 06:56:51,


In [25]:
req = bv.inbox()[0]

In [26]:
req

BeaverEnvelope [CODE]: real_result
  From: alice
  Type: ComputationRequest (__main__)
  Size: 2173 bytes
  Created: 2025-11-20 06:56:51 UTC
  ID: 45e80be3...

Arguments (1):
  [0] 🌍 PUBLIC Twin: patient_data (type: Twin[DataFrame], owner: bob)

Signature: real_result(df)

Source:
  def analyze_patients(df):
      """Analysis function developed on mock data."""
      results = {
          'total_patients': len(df),
          'avg_age': df['age'].mean(),
          'avg_test_result': df['test_result'].mean(),
          'positive_count': (df['diagnosis'] == 'positive').sum(),
          'positive_rate': (df['diagnosis'] == 'positive').sum() / len(df) * 100
      }
      return results

In [27]:
req.load()

✓ Loaded 'real_result' into globals


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
⚡ Action: analyze_patients_result
   Request from: alice
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📋 Function: analyze_patients

📦 Bound Data:
  │ 🌍 PUBLIC
  │   Parameter: patient_data
  │   Type: DataFrame
  │   Owner: bob
  │   📊 Mock data available for testing

💡 Next Steps:
   .run_mock()   → Test on mock/public data (safe preview)
   .run()        → Execute on real/private data
   .run_both()   → Run on both mock & real for comparison
   .reject(msg)  → Decline this request

🆔 IDs: comp=358311c0739d... result=285db3b082f7...
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [28]:
real_result

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
⚡ Action: analyze_patients_result
   Request from: alice
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📋 Function: analyze_patients

📦 Bound Data:
  │ 🌍 PUBLIC
  │   Parameter: patient_data
  │   Type: DataFrame
  │   Owner: bob
  │   📊 Mock data available for testing

💡 Next Steps:
   .run_mock()   → Test on mock/public data (safe preview)
   .run()        → Execute on real/private data
   .run_both()   → Run on both mock & real for comparison
   .reject(msg)  → Decline this request

🆔 IDs: comp=358311c0739d... result=285db3b082f7...
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [29]:
r = real_result.run_both()

🧪 Step 1/2: Testing on mock/public data...
🧪 Testing on mock/public data...
✓ Mock result: dict
🔒 Step 2/2: Executing on real/private data...
⚙️  Executing: analyze_patients_result = analyze_patients(...)
   From: alice
   Computation ID: 358311c0739d...
   Context: bob
✓ Execution complete
✓ Result: dict


In [30]:
r

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ComputationResult: analyze_patients_result
  Request from: alice
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

⚠️  Twin: analyze_patients_result (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    {'total_patients': 10, 'avg_age': np.float64(40.9), 'avg_...    ← .value uses this
  🌍 Public    {'total_patients': 5, 'avg_age': np.float64(40.0), 'avg_t...    ✓
  Live: ⚫ Disabled
  IDs: twin=6ce35b36... private=ffe11257... public=1f6a9db8...

  📤 Output captured during execution:
     🔒 Using PRIVATE data from Twin 'patient_data...'

  💡 Actions:
     .approve()           - Send private/real result back
     .approve_mock()      - Send mock result (continue iteration)
     .reject(message)     - Reject with explanation
     .data                - Access Twin to inspect/modify

In [31]:
r.result

⚠️  Twin: analyze_patients_result (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    {'total_patients': 10, 'avg_age': np.float64(40.9), 'avg_...    ← .value uses this
  🌍 Public    {'total_patients': 5, 'avg_age': np.float64(40.0), 'avg_t...    ✓
  Live: ⚫ Disabled
  IDs: twin=6ce35b36... private=ffe11257... public=1f6a9db8...

In [32]:
r.approve()

✅ Approving private/real result for: analyze_patients_result
✅ Approving result for: analyze_patients_result
   Sending to: alice
✓ Result sent to alice's inbox
💡 They can access it with bv.inbox()['analyze_patients_result'].load()


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📤 Send Result
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📧 Envelope ID: 8dff4aaad8f240d0...
📝 Name: analyze_patients_result
👤 Sender: bob → Recipient: alice
📁 File: shared/alice/8dff4aaad8f240d08a06145f93974f1b.beaver
📦 Type: dict (data)

✅ Successfully sent to alice's inbox
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━